# Bibliotecas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans

# Dados

In [ ]:
df_ies = pd.read_excel("Base para Clusterizacao.xlsx")

In [ ]:
df_ies.head()

In [ ]:
df_variaveis = df_ies[["Quant Nº Alunos", "Quant N° Curso", "População", "median(NU_IDADE)", "Evasão", "CP", "RM", "CI"]]

In [ ]:
df_variaveis["median(NU_IDADE)"] = df_variaveis["median(NU_IDADE)"].astype(str).str.replace(",", ".").astype(float)

# Verificação de correlação entre variáveis

In [ ]:
corr = df_variaveis.corr()
corr.style.background_gradient(cmap='coolwarm')

## Eliminação de variáveis correlacionadas

In [ ]:
df_variaveis = df_variaveis.drop(["CP", "RM", "CI"], 1)

In [ ]:
corr = df_variaveis.corr()
corr.style.background_gradient(cmap='coolwarm')

# Padronização das variáveis

In [ ]:
df_variaveis_std = StandardScaler().fit_transform(df_variaveis)

# Verificação do número de clusters ideal

## Método do cotevelo

In [ ]:
for x in (2**exp for exp in range(2, 9)):
    print(x)

In [ ]:
wcss = []
for n_clusters in (2**exp for exp in range(0, 7)):
    kmeans = KMeans(n_clusters = n_clusters, init = 'k-means++', random_state = 42)
    kmeans.fit(df_variaveis_std)
    wcss.append(kmeans.inertia_)
plt.plot([2**exp for exp in range(0, 7)], wcss)
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')
plt.show()

## Método da silhueta

In [ ]:
for i, k in enumerate([4, 5, 6, 7, 8]):
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(18, 7)
    
    # Run the Kmeans algorithm
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(df_variaveis_std)
    print("Número de clusters: ", k)
    print("Inércia: ", round(kmeans.inertia_, 2))
    # Get silhouette samples
    silhouette_vals = silhouette_samples(df_variaveis_std, labels)

    # Silhouette plot
    y_ticks = []
    y_lower, y_upper = 0, 0
    for i, cluster in enumerate(np.unique(labels)):
        cluster_silhouette_vals = silhouette_vals[labels == cluster]
        cluster_silhouette_vals.sort()
        y_upper += len(cluster_silhouette_vals)
        ax1.barh(range(y_lower, y_upper), cluster_silhouette_vals, edgecolor='none', height=1)
        ax1.text(-0.03, (y_lower + y_upper) / 2, str(i + 1))
        y_lower += len(cluster_silhouette_vals)

    # Get the average silhouette score and plot it
    score_medio = np.mean(silhouette_vals)
    print("Score médio: ", round(score_medio, 2))
    ax1.axvline(score_medio, linestyle='--', linewidth=2, color='green')
    ax1.set_yticks([])
    ax1.set_xlim([-1, 1])
    ax1.set_xlabel('Silhouette coefficient values')
    ax1.set_ylabel('Cluster labels')
    ax1.set_title('Silhouette plot for the various clusters', y=1.02);
    plt.show()
    print()

# Verificação da variação dos resultados

In [ ]:
wcss = []
n_verificacoes = 200
for i in range(1, n_verificacoes):
    kmeans = KMeans(n_clusters = 6, init = 'k-means++')
    kmeans.fit(df_variaveis_std)
    wcss.append(kmeans.inertia_)
print("Menor valor de inércia: ", round(min(wcss), 2))
print()
plt.bar(range(1, n_verificacoes), wcss-min(wcss), bottom=min(wcss))
plt.xlabel('Verificação')
plt.ylabel('WCSS')
plt.show()

# Clusterização

In [ ]:
kmeans = KMeans(n_clusters = 6, init = 'k-means++')
y_kmeans_0 = kmeans.fit_predict(df_variaveis_std)
print("Inérica: ", round(kmeans.inertia_, 2))

In [ ]:
kmeans = KMeans(n_clusters = 6, init = 'k-means++')
y_kmeans_1 = kmeans.fit_predict(df_variaveis_std)
print("Inérica: ", round(kmeans.inertia_, 2))

# Análise de consistência da clusterização

In [ ]:
pd.crosstab(y_kmeans_0, y_kmeans_1)

# Marcação dos clusters

In [ ]:
df_result = df_ies.merge(pd.DataFrame(y_kmeans_0), how="left", left_index=True, right_index=True)

# Resultado

In [ ]:
df_result.to_excel("result.xlsx")